#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2018-03-29 11:07:15,506] Making new env: FrozenLake-v0


In [2]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [4]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 1
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [5]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [6]:
env.reset()
new_obs, reward, is_done, _ = env.step(action_to_i['right'])
print(new_obs)
env.render()

1
  (Right)
SFFF
FHFH
FFFH
HFFG


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [7]:
import numpy as np
import random

n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    a = np.array([np.random.choice(range(n_actions)) for i in range(n_states)])
    
    return a

In [8]:
a = get_random_policy()

In [9]:
a

array([3, 3, 3, 2, 1, 0, 3, 3, 0, 2, 0, 0, 2, 3, 3, 1])

In [10]:
policies = [get_random_policy() for i in range(10**4)]
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.25123125  0.2501125   0.24990625  0.24875   ]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [11]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    old_obs = 0
    total_reward = 0
    for i in range(t_max):
        new_obs, reward, is_done, _ = env.step(policy[old_obs])
        old_obs = new_obs
        total_reward += reward
        if is_done:
            break

    return total_reward

In [13]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]

generating 10^3 sessions...


In [14]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env,get_random_policy()) for _ in range(n_times)]
    return float(np.mean(rewards))
        

In [17]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way."""
    lake = "SFFFFHFHFFFHHFFG"

    # where to move from each tile (we're a bit unsure if this is accurate)
    arrows = ['<v>^'[a] for a in policy]
    
    # draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow, tile in zip(arrows, lake)]
    
    for i in range(0, 16, 4):
        print(' '.join(signs[i:i+4]))

print("random policy:")
print_policy(get_random_policy())

random policy:
> > > >
> H < H
^ v < H
H ^ < G


### Main loop

In [18]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
        print("Best policy:")
        print_policy(best_policy)

  0%|          | 5/10000 [00:00<08:32, 19.51it/s]

New best score: 0.01
Best policy:
^ < > ^
v H > H
> > ^ H
H ^ < G
New best score: 0.03
Best policy:
< v v <
v H < H
^ ^ ^ H
H < < G


  0%|          | 14/10000 [00:00<07:06, 23.40it/s]

New best score: 0.04
Best policy:
^ v < <
> H > H
v v > H
H < v G


  1%|          | 110/10000 [00:04<05:22, 30.64it/s]

...

  2%|▏         | 232/10000 [00:10<07:54, 20.58it/s]

New best score: 0.07
Best policy:
> ^ < <
> H < H
^ < < H
H v ^ G


 16%|█▌        | 1568/10000 [01:07<05:13, 26.93it/s]

New best score: 0.08
Best policy:
v > < ^
> H v H
v > ^ H
H ^ v G


100%|██████████| 10000/10000 [06:11<00:00, 30.34it/s]


# Part II Genetic algorithm

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [19]:
from random import random, randrange

In [20]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    crossover_policy = []
    for state1, state2 in zip(policy1, policy2):
        if np.random.uniform() <= p:
            crossover_policy.append(state1)
        else:
            crossover_policy.append(state2)
    return np.array(crossover_policy)

In [21]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """    

    return crossover(policy, get_random_policy(), p=p)
    

In [22]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

In [23]:
n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick

In [24]:
print("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
pool_scores = [evaluate(i) for i in pool]

initializing...


In [25]:
#main loop
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    crossovered = [crossover(pool[randrange(len(pool))], pool[randrange(len(pool))]) for _ in range(n_crossovers)]
    mutated = [mutation(pool[randrange(len(pool))]) for _ in range(n_mutations)]
        
    #add new policies to the pool
    pool = pool + crossovered + mutated
    pool_scores = [evaluate(i) for i in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])
    print_policy(pool[-1])

Epoch 0:
best score: 0.08
v > < <
^ H ^ H
> ^ < H
H > < G
Epoch 1:
best score: 0.06
> v v v
v H < H
v ^ < H
H > ^ G
Epoch 2:
best score: 0.05
< v < >
< H > H
> v > H
H v ^ G
Epoch 3:
best score: 0.07
< v < v
v H ^ H
v > ^ H
H ^ ^ G
Epoch 4:
best score: 0.06
v > v >
^ H < H
v > < H
H < v G
Epoch 5:
best score: 0.07
^ < ^ v
v H v H
v < < H
H > v G
Epoch 6:
best score: 0.05
v v v v
< H v H
> < v H
H > ^ G
Epoch 7:
best score: 0.06
< > > ^
> H v H
^ > ^ H
H v > G
Epoch 8:
best score: 0.06
v v v <
v H < H
^ ^ < H
H ^ ^ G
Epoch 9:
best score: 0.05
< ^ v v
^ H < H
> v < H
H < < G
Epoch 10:
best score: 0.06
< ^ v >
> H ^ H
^ ^ v H
H > ^ G
Epoch 11:
best score: 0.06
> ^ > ^
^ H > H
< > > H
H ^ v G
Epoch 12:
best score: 0.06
< > < <
v H ^ H
^ > v H
H ^ > G
Epoch 13:
best score: 0.05
^ < v <
^ H > H
v ^ v H
H ^ < G
Epoch 14:
best score: 0.06
< ^ v >
v H > H
^ v < H
H v v G
Epoch 15:
best score: 0.05
> > > >
> H > H
v ^ v H
H < < G
Epoch 16:
best score: 0.06
< > < ^
^ H < H
> ^ > H
H > < G
Epoch 1